In [1]:

#import libraries and frameworks
from typing import Tuple, List

import numpy as np
import open3d as o3d
import os
import copy
import numpy as np
import matplotlib.pyplot as plt
import exercises.tools.utils as utils
from matplotlib import animation, rc
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from IPython.display import display, Math, Latex, Markdown, HTML

from tqdm import tqdm
import time
import pandas as pd

%matplotlib inline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## NDT

The normal distribution transform calculates the probability densitiy function based on the normal distribution for the points to estimatate the likelyhood of sampling a point at given area in the pointcloud, this is done piecewise by dividing the pointcloud map into a grid. that a point belongs into certain grid cell. The transformation from one pointcloud to another becomes a statistical problem and
can now be solved using e.g.: maximum likelyhood optimization. One possibility is to use the newton alogrithm to optimize the function
and find the transform.

### Module
In this implementation the poincloud library (https://pointclouds.org/) was used, with python bindings from (https://github.com/hummat/registration).
The bindings were slightly adopted for our use case and the pcl110 registration library was used.

The module uses mostly default parameter except the following:
- epsilon: was chosen for 0.001 to be in line with the assignment
- downsample: was chosen as 0.5 to speed up the transform calculation
- voxelize: is set to 0.2 to help again with computation time

The distance threshold and inlier threshold had no measerable effect on the performance when tuning it within reasonable ranges.

In [2]:
#lib wrapper for cpp registration libary slightly altered from: https://github.com/hummat/registration
#using registration library pcl110

import os
import ctypes
import csv

import numpy as np


def load_library(path: str = os.getcwd(), name: str = "libregistration_pcl110") -> None:
    global REGLIB
    try:
        REGLIB = np.ctypeslib.load_library(libname=name, loader_path=path)
        print("REGLIB loaded")
        print(REGLIB)
    except OSError:
        print("Compiled C++ library was not found in the current directory. Please use `load_library` to load it from "
              "a custom directory, then ignore this message.")


load_library()


def load_data(path: str, delimiter: str = ' ') -> np.ndarray:
    """Loads point cloud data of type `CSV`, `PLY` and `PCD`.

    The file should contain one point per line where each number is separated by the `delimiter` character.
    Any none numeric lines will be skipped.

    Args:
        path (str): The path to the file.
        delimiter (char): Separation of numbers in each line of the file.

    Returns:
        A ndarray of shape NxD where `N` are the number of points in the point cloud and `D` their dimension.
    """
    data = list()
    with open(path, newline='\n') as file:
        reader = csv.reader(file, delimiter=delimiter, quoting=csv.QUOTE_NONNUMERIC)
        lines = 0
        skips = 0
        while True:
            try:
                row = next(reader)
                row = [x for x in row if not isinstance(x, str)]
                if len(row) in [3, 6, 9]:
                    data.append(row[:3])
                else:
                    skips += 1
            except ValueError:
                skips += 1
                pass
            except StopIteration:
                print(f"Found {lines} lines. Skipped {skips}. Loaded {lines - skips} points.")
                break
            lines += 1
    return np.array(data)


def set_argtypes(algorithm, source, target):
    """Tells the underlying C++ code which data types and dimensions to expect.

    Args:
        algorithm (str): The registration algorithm to use. One of `icp` or `ndt`.
        source (ndarray): The source point cloud.
        target (ndarray): The target point cloud.
    """
    REGLIB.icp.restype = ctypes.c_double
    REGLIB.ndt.restype = ctypes.c_double
    argtypes = [np.ctypeslib.ndpointer(dtype=np.float64, ndim=source.ndim, shape=source.shape,
                                       flags='C_CONTIGUOUS'), ctypes.c_size_t,
                np.ctypeslib.ndpointer(dtype=np.float64, ndim=target.ndim, shape=target.shape,
                                       flags='C_CONTIGUOUS'), ctypes.c_size_t,
                np.ctypeslib.ndpointer(dtype=np.float64, ndim=2, shape=(4, 4), flags='C_CONTIGUOUS'),
                ctypes.c_int, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_bool]
    if algorithm == 'icp':
        REGLIB.icp.argtypes = argtypes
    elif algorithm == 'ndt':
        argtypes.extend([ctypes.c_float, ctypes.c_double, ctypes.c_float])
        REGLIB.ndt.argtypes = argtypes


def icp(source,
        target,
        transformation,
        nr_iterations=25,
        distance_threshold=1.0,
        epsilon=1e-6,
        inlier_threshold=0.0005,
        downsample=0,
        visualize=False):
    """The `Iterative Closest Point` (ICP) algorithm.

    Args:
        source (ndarray): The point cloud that we want to align to the target.
        target (ndarray): The point cloud that the source is aligned to.
        nr_iterations (int): The maximum number of iterations the internal optimization should run for.
        distance_threshold (float): The maximum distance threshold between two correspondent points in
                                    source -> target. If the distance is larger than this threshold, the points will
                                    be ignored in the alignment process.
        epsilon (float): The transformation epsilon (maximum allowable difference between two consecutive
                 transformations) in order for an optimization to be considered as having converged to the final
                 solution.
        inlier_threshold (float): The inlier distance threshold for the internal RANSAC outlier rejection loop.
                          The method considers a point to be an inlier, if the distance between the target data
                          index and the transformed source index is smaller than the given inlier distance
                          threshold.
        downsample (float): Assembles a local 3D grid over a given PointCloud and downsamples + filters the data.
        visualize (bool): Can be used to visualize and control the progress of the algorithm.

    Returns:
        A ndarray with the final transformation matrix between source and target.
    """

 
    
    set_argtypes('icp', source, target)
    score = REGLIB.icp(source, len(source), target, len(target), transformation,
                       nr_iterations, distance_threshold, epsilon, inlier_threshold, downsample, visualize)
    print(f"ICP converged. Fitness score: {score:.2f}") if score > 0 else print("ICP did not converge!")
    return transformation


def ndt(source,
        target,
        nr_iterations=25,
        distance_threshold=1.0,
        epsilon=0.01,
        inlier_threshold=0.05,
        downsample=0,
        visualize=False,
        resolution=1.0,
        step_size=0.1,
        voxelize=0,
        transformation=np.identity(4)):
    """The `Normal Distributions Transform` (NDT) algorithm.

    Args:
        source (ndarray): The point cloud that we want to align to the target.
        target (ndarray): The point cloud that the source is aligned to.
        nr_iterations (int): The maximum number of iterations the internal optimization should run for.
        distance_threshold (float): The maximum distance threshold between two correspondent points in
                                    source -> target. If the distance is larger than this threshold, the points will
                                    be ignored in the alignment process.
        epsilon (float): The transformation epsilon (maximum allowable difference between two consecutive
                 transformations) in order for an optimization to be considered as having converged to the final
                 solution.
        inlier_threshold (float): The inlier distance threshold for the internal RANSAC outlier rejection loop.
                          The method considers a point to be an inlier, if the distance between the target data
                          index and the transformed source index is smaller than the given inlier distance
                          threshold.
        downsample (float): Assembles a local 3D grid over a given PointCloud and downsamples + filters the data.
        visualize (bool): Can be used to visualize and control the progress of the algorithm.
        resolution (float): The resolution of the voxel grid. Try increasing this in case of core dumps.
        step_size (float): The Newton line search maximum step length.
        voxelize (bool): If set to `True`, the source cloud is converted into a voxel model before alignment.

    Returns:
        A ndarray with the final transformation matrix between source and target.
    """


    set_argtypes('ndt', source, target)
    score = REGLIB.ndt(source, len(source), target, len(target), transformation,
                  nr_iterations, distance_threshold, epsilon, inlier_threshold, downsample, visualize,
                  resolution, step_size, voxelize)
    #print(f"NDT converged. Fitness score: {score:.2f}") if score > 0 else print("NDT did not converge!")
    return transformation, score

Compiled C++ library was not found in the current directory. Please use `load_library` to load it from a custom directory, then ignore this message.


In [3]:
#implementation of voxel filter using open3d to downsample points
def voxel_downsample(points, voxel_size):

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)


    frame_cloud_downsampled = pcd.voxel_down_sample(voxel_size)

    return np.array(frame_cloud_downsampled.points)

In [7]:
#Load source frames, target map and ground truth
frames_dir = "/app/dataset/frames"
frame_files = os.listdir(frames_dir)
#frames = [o3d.io.read_point_cloud(os.path.join(frames_dir, f)) for f in frame_files]


#load ground truth csv as pandas dataframe
g_df = pd.read_csv("/app/dataset/ground_truth.csv")
samples = len(frame_files)


#Load target map points
target_points = load_data("/app/dataset/map.pcd")
load_library("app/libregistration_pcl110.so")

Found 143960 lines. Skipped 11. Loaded 143949 points.
REGLIB loaded
<CDLL '/app/libregistration_pcl110.so', handle 5503cf0 at 0x7f401a49a340>


## ICP localization ##

In [8]:
#initial transformation
initTransform = np.identity(4)
icp_errors =[]
voxel_size = 0.2

start_time = time.time()

#loops over all frames and returns the the lateral errors 
for sample in tqdm(range(samples)):

    frame_path = os.path.join(frames_dir, frame_files[sample])
    source_points = load_data(frame_path)
    
    
    #voxel filter
    points = voxel_downsample(source_points, 0.2)


    #transfrom source points based on last alignment transformation
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd = pcd.transform(initTransform)
    t_points = np.array(pcd.points)

    #ICP alignmnet returns transformation matrix


    
    trans= icp(source= t_points, target=target_points,transformation=np.identity(4), nr_iterations=30, epsilon=1e-6,
                         inlier_threshold=1e-6, distance_threshold= 1, downsample = 0, visualize=False)
    

    initTransform = trans * initTransform

    
    #grab frame values from ground truth
    frame_gt = g_df[g_df['Frame'] == sample]


    
    # Calculate L2 norm for latent error 
    errors = np.linalg.norm(np.array([initTransform[0,3],initTransform[1,3],initTransform[2,3]])-np.array([ frame_gt[' x'].item(),  frame_gt[' y'].item(),  frame_gt[' z'].item()]))  
    print('Frame :', sample)
    print("Error--------:", errors) 
    if errors>1.2:
        print("Latent error above the threshold of 1.2")

  
    initTransform = trans 

    icp_errors.append(errors)

end_time = time.time()
total_icp_time = end_time - start_time

print(f"Total execution time: {total_icp_time:.3f} seconds")



  0%|          | 0/1014 [00:00<?, ?it/s]

Found 9104 lines. Skipped 11. Loaded 9093 points.


  0%|          | 1/1014 [00:05<1:32:47,  5.50s/it]

ICP converged. Fitness score: 0.08
Frame : 0
Error--------: 0.0
Found 22369 lines. Skipped 11. Loaded 22358 points.


  0%|          | 2/1014 [00:06<51:59,  3.08s/it]  

...

ICP converged. Fitness score: 4.25
Frame : 1011
Error--------: 182.38717359308922
Latent error above the threshold of 1.2
Found 26254 lines. Skipped 11. Loaded 26243 points.


100%|█████████▉| 1013/1014 [15:06<00:00,  1.10it/s]

ICP converged. Fitness score: 6.70
Frame : 1012
Error--------: 182.37863381375402
Latent error above the threshold of 1.2
Found 12734 lines. Skipped 11. Loaded 12723 points.


100%|██████████| 1014/1014 [15:07<00:00,  1.12it/s]

ICP converged. Fitness score: 4.29
Frame : 1013
Error--------: 182.3788015760444
Latent error above the threshold of 1.2
Total execution time: 907.065 seconds


## NDT LOCALIZATION ##

In [9]:
#initial transformation
initTransform = np.identity(4)
ndt_errors =[]
voxel_size = 0.2

start_time = time.time()

#loops over all frames and returns the the lateral errors 
for sample in tqdm(range(samples)):

    frame_path = os.path.join("/app/dataset/frames/", frame_files[sample])
    source_points = load_data(frame_path)
    
    
    #voxel filter
    points = voxel_downsample(source_points, 0.2)


    #transfrom source points based on last alignment transformation
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd = pcd.transform(initTransform)
    t_points = np.array(pcd.points)

    #NDT alignmnet returns transformation matrix

    trans= ndt(source= t_points, target=target_points,transformation=np.identity(4),voxelize=0, nr_iterations=60, epsilon=1e-3,
                         inlier_threshold=1e-3, distance_threshold= 1, downsample = 0, visualize=False)
    

    initTransform = trans 

    
    #grab frame values from ground truth
    frame_gt = g_df[g_df['Frame'] == sample]


    
    # Calculate L2 norm for latent error 
    errors = np.linalg.norm(np.array([initTransform[0,3],initTransform[1,3],initTransform[2,3]])-np.array([ frame_gt[' x'].item(),  frame_gt[' y'].item(),  frame_gt[' z'].item()]))  
    print('Frame :', sample)
    print("Error--------:", errors) 
    if errors>1.2:
        print("Too big error")


    #initTransform = trans 

    ndt_errors.append(errors)

end_time = time.time()
total_ndt_time = end_time - start_time

print(f"Total execution time: {total_ndt_time:.3f} seconds")


  0%|          | 0/1014 [00:00<?, ?it/s]

Found 9104 lines. Skipped 11. Loaded 9093 points.


  0%|          | 0/1014 [03:38<?, ?it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

## Analysis 

### RMSE

### Runtime
Without any downsampling the *ndt* needs around 15 seconds for the calculation of one transform from our data set. With the chosen downsampling 
and voxelization parameter, the calculation takes only roughly one second. While trying the algorithm on not downsampled data samples,
no performance difference could be observed.

In [ ]:
error_icp = np.array(icp_errors)
Mean_icp_error = np.mean(error_icp)
Max_icp_error = np.max(error_icp)

print("Number of sucessful frames localization using ICP :", len(error_icp[error_icp<1.2]))
print("Mean error using ICP localization :",f'{Mean_icp_error:.3f}')
print("Max error using ICP localization :", f'{Max_icp_error:.3f}')
print("Time needed for localization of all frames using ICP :", f'{total_icp_time:.3f}', "seconds")


Number of sucessful frames localization using ICP : 32
Mean error using ICP localization : 10.237
Max error using ICP localization : 24.898


In [ ]:
error_ndt = np.array(ndt_errors)
Mean_ndt_error = np.mean(error_ndt)
Max_ndt_error = np.max(error_ndt)

print("Number of failed frames localization using NDT :", len(error_ndt[error_ndt<1.2]))
print("Mean error using NDT localization :", f'{Mean_ndt_error:.3f}')
print("Max error using NDT localization :", f'{Max_ndt_error:.3f}')
print("Time needed for localization of all frames using ndt :", f'{total_ndt_time:.3f}', "seconds")